In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, d2_pinball_score
import copy
from numpy import array, asarray
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing
import random
from statistics import mean
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

np.set_printoptions(precision = 3, floatmode='fixed', suppress='True')

# загрузка данных
read_data = pd.read_excel("данные3.xlsx")

data = pd.DataFrame({"date":read_data["Date"], "temperature":read_data["Daily minimum temperatures"]})

data['temperature'] = data['temperature'].astype(float)

# создадим временной ряд с индексами из дат
data_indexed = copy.deepcopy(data)
data_indexed.set_index('date', inplace = True)
data_indexed.index = pd.to_datetime(data_indexed.index)

# визуализация временного ряда
plt.xlabel('Время')
plt.ylabel('Минимальная температура')
plt.plot(data_indexed, color="maroon" )
plt.title('Минимальная температура с 1981 по 1990 годы')
plt.show()

# разложим модель на сезонность + тренд + остатки, данные есть каждый день
# период - 365, так как есть сезонность (ежегодная), так как это температура
decompose = seasonal_decompose(data_indexed, model='additive', period=365)

In [ ]:
# методы машинного обучения

# тестовая выборка
m = 365

# определим вход и выход
n_in = 150
n_out = 1

n_in2 = 6 # для внутреннего шага и s = 1 (для m = 365)
inside_step = m

n_in3 = 15 # для внутреннего шага и s > 1 (для m = 365)

inside_step2 = m + 1

# определим s
s1 = 2

n_out2 = m # для множественного выхода


# разделим одномерный ряд на сэмплы с внутренним шагом = 1
def split_sequence1(sequence, n_steps_in, n_steps_out):
 X, y = list(), list()
 for i in range(len(sequence)):
  # найдём конец сэмпла
  end_ix = i + n_steps_in
  out_end_ix = end_ix + n_steps_out
  # проверим, что мы не вышли за пределы сэмпла
  if out_end_ix > len(sequence):
    break
  # соберём входные и выходные части
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

# разделим одномерный ряд на сэмплы с внутренним шагом = m
def split_sequence2(sequence, n_steps_in, n_steps_out, inside_step):
 X, y = list(), list()
 for i in range(len(sequence)):
  # найдём конец сэмпла
  end_ix = i + inside_step*(n_steps_in-1)
  out_end_ix = end_ix + n_steps_out*inside_step
  # проверим, что мы не вышли за пределы ряда
  if (out_end_ix >= len(sequence)):
    break
  # соберём входные и выходные части
  seq_x = []
  seq_y = []
  for j in range(n_steps_in):
    seq_x.append(sequence[i+j*inside_step])
  for j in range(n_steps_out):
    seq_y.append(sequence[out_end_ix])
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

# разделим одномерный ряд на сэмплы с внутренним шагом = m+1 и s > 1
def split_sequence3(sequence, n_steps_in, n_steps_out, inside_step, s):
 X, y = list(), list()
 for i in range(len(sequence)):
  # найдём конец сэмпла
  end_ix = i + inside_step*(n_steps_in/s - 1) + s - 1
  out_end_ix = (end_ix - s + 1) + n_steps_out*inside_step
  # проверим, что мы не вышли за пределы ряда
  if (out_end_ix >= len(sequence)):
    break
  # соберём входные и выходные части
  seq_x = []
  seq_y = []
  for j in range(int(n_steps_in/s)):
    r = int(i+j*inside_step)
    q = int(i+j*inside_step+s-1)
    for z in range(q-r+1):
      seq_x.append(sequence[r+1*z])
  for j in range(n_steps_out):
    t = int(out_end_ix)
    seq_y.append(sequence[t])
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

# одинаковые результаты для каждого запуска
seed_value = 0
np.random.seed(seed_value)
random.seed(seed_value)

# рекурсивная стратегия:

# определим тренировочные входные данные
raw_seq = observed
# выберем размер входных и выходных горизонтов
n_steps_in, n_steps_out = n_in, n_out

# разобъём на сэмплы входные и выходные данные (с внутренним шагом = 1)
X, y = split_sequence1(raw_seq, n_steps_in, n_steps_out)

# изменим форму на [сэмплы, шаг]
X = X.reshape((X.shape[0], X.shape[1]))
y = y.reshape((y.shape[0], y.shape[1]))

# количество сэмплов
k = len(y)

# создание тренировочной выборки (в ней нет тестовой выборки)
l = k - m

# создание тренировочной выборки
X_train1 = X[:l]
y_train1 = y[:l]

# создание тестовой выборки
X_test1 = X[l:]
y_test1 = y[l:]

# особенное разделение на сэмплы, s = 1:

# определим тренировочные входные данные
raw_seq = observed
# выберем размер входных и выходных горизонтов
n_steps_in, n_steps_out = n_in2, n_out

# разобъём на сэмплы входные и выходные данные (с внутренним шагом = m)
X, y = split_sequence2(raw_seq, n_steps_in, n_steps_out, inside_step)

# изменим форму на [сэмплы, шаг]
X = X.reshape((X.shape[0], X.shape[1]))
y = y.reshape((y.shape[0], y.shape[1]))

# количество сэмплов
k = len(y)

# создание тренировочной выборки (в ней нет тестовой выборки)
l = k - m

# создание тренировочной выборки
X_train2 = X[:l]
y_train2 = y[:l]

# создание тестовой выборки
X_test2 = X[l:]
y_test2 = y[l:]

# множественный выход:

# определим тренировочные входные данные
raw_seq = observed
# выберем размер входных и выходных горизонтов
n_steps_in, n_steps_out = n_in, n_out2

# разобъём на сэмплы входные и выходные данные с внутренним шагом = m и s > 1
X, y = split_sequence1(raw_seq, n_steps_in, n_steps_out)

# изменим форму на [сэмплы, шаг]
X = X.reshape((X.shape[0], X.shape[1]))
y = y.reshape((y.shape[0], y.shape[1]))

# количество сэмплов
k = len(y)

# создание тренировочной выборки (в ней нет тестовой выборки)
l = k - m

# создание тренировочной выборки
X_train6 = X[:l]
y_train6 = y[:l]

# создание тестовой выборки
X_test6 = X[l:]
y_test6 = y[l:]

test_dates = observed[-m:].index

y_show = copy.deepcopy(observed[:len(observed)-m])
plt.plot(y_show, label = 'Тренировочная выборка', color='royalblue')
plt.plot(test_dates, y_test1, label = 'Тестовая выборка', color='black')
plt.legend(loc='center right', bbox_to_anchor=(1.48, 0.5))
plt.show()

In [ ]:
# одинаковые результаты для каждого запуска
seed_value = 0
np.random.seed(seed_value)
random.seed(seed_value)

print("Extra Trees:")

# рекурсия

# определим модель
model5 = ExtraTreesRegressor(n_jobs = -1, random_state = 0)

# подгонка тренировочных данных
model5.fit(X_train1, np.ravel(y_train1))

# прогнозируем
yhat_test1 = []
x_input = copy.deepcopy(X_test1[0:1])

# прогнозируем значения по одному, основывыясь на прогнозах
for i in range(len(y_test1)):
  # делаем прогноз на один элемент вперед
  yi = model5.predict(x_input)
  yhat_test1.append(yi.flatten())
  # удаляем первое значение
  x_input = np.delete(x_input[0], 0)
  # добавляем в конец спрогнозированное значение
  x_input = np.append(x_input, yi)
  # поправляем форму для дальнейших прогнозов
  x_input = x_input.reshape((1, n_in))

# оценим модель
r1 = metrics.r2_score(y_test1, yhat_test1)
mae1 = metrics.mean_absolute_error(y_test1, yhat_test1)
d1 = metrics.d2_pinball_score(y_test1, yhat_test1)
mse1 = metrics.mean_squared_error(y_test1, yhat_test1)
rmse1 = np.sqrt(mse1)

print("\nРекурсивная стратегия:")
print('R2 =', '{:.3f}'.format(r1))
print('RMSE =', '{:.3f}'.format(rmse1))
print('MAE =', '{:.3f}'.format(mae1))
print('D2_pinball =', '{:.3f}'.format(d1))

# особенное разделенеи на сэмплы

# определим модель
model5 = ExtraTreesRegressor(n_jobs = -1, random_state = 0)
# подгонка тренировочных данных
model5.fit(X_train2, np.ravel(y_train2))

# прогнозируем
yhat_test2 = model5.predict(X_test2)

# оценим модель
r2 = metrics.r2_score(y_test2, yhat_test2)
mae2 = metrics.mean_absolute_error(y_test2, yhat_test2)
d2 = metrics.d2_pinball_score(y_test2, yhat_test2)
mse2 = metrics.mean_squared_error(y_test2, yhat_test2)
rmse2 = np.sqrt(mse2)

print("\nСэмплирование с особенным разделением на сэмплы, s = 1:")
print('R2 =', '{:.3f}'.format(r2))
print('RMSE =', '{:.3f}'.format(rmse2))
print('MAE =', '{:.3f}'.format(mae2))
print('D2_pinball =', '{:.3f}'.format(d2))


# множественный выход

# определим модель
model5 = ExtraTreesRegressor(n_jobs = -1, random_state = 0)

# подгонка тренировочных данных
model5.fit(X_train6, y_train6)

# прогнозируем
yhat_test6 = []

x_input = copy.deepcopy(X_test6[-1])
x_input = x_input.reshape((1, n_in))

yhat_test6 = model5.predict(x_input)

yhat_test6 = yhat_test6.flatten()

# оценим модель
r6 = metrics.r2_score(y_test6[-1], yhat_test6)
mae6 = metrics.mean_absolute_error(y_test6[-1], yhat_test6)
d6 = metrics.d2_pinball_score(y_test6[-1], yhat_test6)
mse6 = metrics.mean_squared_error(y_test6[-1], yhat_test6)
rmse6 = np.sqrt(mse6)

print("\nСтратегия с множественным выходом:")
print('R2 =', '{:.3f}'.format(r6))
print('RMSE =', '{:.3f}'.format(rmse6))
print('MAE =', '{:.3f}'.format(mae6))
print('D2_pinball =', '{:.3f}'.format(d6))


# визуализация
print()

plt.plot(y_test1, label = 'Тестовая выборка', color='black', marker = '*', linewidth = 1.8)
plt.plot(yhat_test1, label = 'Рекурсия', color='lime')
plt.plot(yhat_test2, label = 'Сэмплирование при s = 1', color='red')
plt.plot(yhat_test6, label = 'Множественный выход', color='darkgreen')
plt.legend(loc='center right', bbox_to_anchor=(1.52, 0.5))
plt.title("Экстремально случайные деревья")
plt.show()